In [1]:
import pandas as pd
import os
import scipy.stats as stats


In [2]:
carpetas = ['GUROBI','CPLEX','MLM','ESGH','LPDH','Configuraciones v2']

In [3]:
gur = []
mlm = []
esgh = []
lpdh = []
cplex = []
configuraciones = []

for c in carpetas:
    archivos = os.listdir(c)
    for ar in archivos: 
        # print(sorted(ar.rstrip(".xlsx").split("_")))
        tiempo, solver, capacidad, nodos  = sorted(ar.rstrip(".xlsx").split("_"))
        tiempo = int(tiempo)
        capacidad = int(capacidad[1:])
        nodos = int(nodos[1:])
        datos = pd.read_excel(f"{c}/{ar}")
        datos["Q"] = capacidad
        datos["nodos"] = nodos
        datos["tiempos"] = tiempo
        if solver == "GUROBI":
            gur.append(datos)
        if solver == "CPLEX":
            cplex.append(datos)
        elif solver == "MLM":
            mlm.append(datos)
        elif solver == "ESGH":
            esgh.append(datos)
        elif solver == "LPDH":
            lpdh.append(datos)
        elif solver in ["Conf1", "Conf2", "Conf3", "Conf4","Conf5"]:
            datos["version"] = "configuracion" + solver[-1]
            configuraciones.append(datos)
        else: pass
        
gurobis = pd.concat(gur)
cplexs  = pd.concat(cplex)
mlms    = pd.concat(mlm)
esghs   = pd.concat(esgh)
lpdhs   = pd.concat(lpdh)
configuraciones   = pd.concat(configuraciones)

In [4]:
gurobis[["LB","UB"]] = gurobis[["LB","UB"]].round(2)
cplexs[["LB","UB"]] = cplexs[["LB","UB"]].round(2)
mlms[["min", "avg"]] = mlms[["min", "avg"]].round(2)
configuraciones[["min", "avg"]] = configuraciones[["min", "avg"]].round(2)

In [5]:
bks = pd.concat((
    mlms.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(), 
    gurobis.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    cplexs.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    esghs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    lpdhs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    configuraciones.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(),
))

bks = bks.groupby(["name", "Q", "nodos"]).agg(bks = ("bks", "min")).reset_index()

gurobis = gurobis.merge(bks, on = ["name", "Q", "nodos"])
cplexs  = cplexs.merge(bks, on = ["name", "Q", "nodos"] )
mlms    = mlms.merge(bks, on = ["name", "Q", "nodos"])
esghs   = esghs.merge(bks, on = ["name", "Q", "nodos"])
lpdhs   = lpdhs.merge(bks, on = ["name", "Q", "nodos"])
configuraciones = configuraciones.merge(bks, on = ["name", "Q", "nodos"])

In [6]:
gurobis["best"] = gurobis["UB"]
cplexs["best"] = cplexs["UB"]
mlms["best"] = mlms["min"]
configuraciones["best"] = configuraciones["min"]
lpdhs["best"] = lpdhs["cost"]
esghs["best"] = esghs["cost"]

gurobis["gap_best"]    = (gurobis["UB"] - gurobis["bks"]) / gurobis["bks"]
cplexs["gap_best"]     = (cplexs["UB"] - cplexs["bks"]) / cplexs["bks"]
mlms["gap_best"]       = (mlms["min"] - mlms["bks"]) / mlms["bks"]
mlms["gap_avg"]        = (mlms["avg"] - mlms["bks"]) / mlms["bks"]
configuraciones["gap_best"]  = (configuraciones["min"] - configuraciones["bks"]) / configuraciones["bks"]
configuraciones["gap_avg"]   = (configuraciones["avg"] - configuraciones["bks"]) / configuraciones["bks"]
lpdhs["gap"]           = (lpdhs["cost"] - lpdhs["bks"]) / lpdhs["bks"]
esghs["gap"]           = (esghs["cost"] - esghs["bks"]) / esghs["bks"]

gurobis["hit"]      =( gurobis["best"] - gurobis["bks"]).apply(lambda x: 1 if x == 0 else 0) 
mlms["hit"]         =( mlms["best"] - mlms["bks"]).apply(lambda x: 1 if x == 0 else 0) 
configuraciones["hit"]    =( configuraciones["best"] - configuraciones["bks"]).apply(lambda x: 1 if x == 0 else 0) 
lpdhs["hit"]        =( lpdhs["best"] - lpdhs["bks"]).apply(lambda x: 1 if x == 0 else 0) 
esghs["hit"]        =( esghs["best"] - esghs["bks"]).apply(lambda x: 1 if x == 0 else 0) 

In [15]:
gurobis

,name,LB,UB,gap,t,Q,nodos,tiempos,bks,best,gap_best,hit
0,rc2_2_10,1506.65,1671.33,0.098532,120.009037,10,150,120,1651.44,1671.33,0.012044,0
1,rc2_2_10,1526.39,1671.33,0.086721,180.007161,10,150,180,1651.44,1671.33,0.012044,0
2,rc2_2_10,1545.69,1651.44,0.064031,3600.007883,10,150,3600,1651.44,1651.44,0.000000,1
3,rc2_2_10,1522.24,1667.91,0.087333,60.006408,10,150,60,1651.44,1667.91,0.009973,0
4,r2_2_6,1568.26,1722.10,0.089332,120.108558,10,150,120,1693.25,1722.10,0.017038,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2955,R203,807.95,842.62,0.041145,60.004840,5,100,60,830.64,842.62,0.014423,0
2956,RC104,940.12,952.69,0.013194,3600.564430,5,100,3600,952.69,952.69,0.000000,1
2957,RC104,917.69,962.51,0.046563,60.013940,5,100,60,952.69,962.51,0.010308,0
2958,R104,801.02,819.58,0.022651,3600.008063,5,100,3600,819.58,819.58,0.000000,1


In [18]:
gurobis_versiones

,name,LB,UB,gap,t,Q,nodos,tiempos,bks,best,gap_best,hit,version
3,rc2_2_10,1522.24,1667.91,0.087333,60.006408,10,150,60,1651.44,1667.91,0.009973,0,gurobi 60
7,r2_2_6,1577.10,1716.29,0.081096,60.006843,10,150,60,1693.25,1716.29,0.013607,0,gurobi 60
11,rc1_2_5,1610.06,1744.78,0.077210,60.311912,10,150,60,1703.20,1744.78,0.024413,0,gurobi 60
15,c1_2_2,1245.42,1395.57,0.107594,61.148333,10,150,60,1337.86,1395.57,0.043136,0,gurobi 60
19,r2_2_8,1436.64,1592.26,0.097732,60.008348,10,150,60,1578.57,1592.26,0.008672,0,gurobi 60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,rc1_2_4,1240.32,1734.34,0.284845,60.035529,20,200,60,1449.11,1734.34,0.196831,0,gurobi 60
2067,rc2_2_3,1306.38,1809.30,0.277961,60.025515,20,200,60,1531.91,1809.30,0.181075,0,gurobi 60
2071,r2_2_5,1777.22,1839.79,0.034013,60.013039,20,200,60,1828.81,1839.79,0.006004,0,gurobi 60
2075,r1_2_6,1600.01,1810.37,0.116201,60.024595,20,200,60,1788.86,1810.37,0.012024,0,gurobi 60


In [24]:
mlms_versiones = mlms[(mlms.tiempos == 60) & (mlms.nodos >= 100)]
mlms_versiones.loc[:,"version"] = "mlm"

gurobis_versiones = gurobis[(gurobis.tiempos == 60) & (gurobis.nodos >= 100)]
gurobis_versiones.loc[:,"version"] = "gurobi60"
gurobis_versiones.loc[:,"t_avg"]   = gurobis_versiones.loc[:,"t"]
gurobis_versiones.loc[:,"gap_avg"]   = gurobis_versiones.loc[:,"gap_best"]

versiones2 = pd.concat((configuraciones , mlms_versiones, gurobis_versiones))
tabla_versiones = versiones2.groupby(["version"]).agg(gap_best  = ("gap_best", "mean"),
                                   gap_avg  =  ("gap_avg", "mean"),
                                   avg_time =  ("t_avg", "mean"),
                                   hit_sum  =  ("hit", "sum")).reset_index()

# tabla_versiones.sort_values(["nodos","version"]).set_index(["nodos", "version"])
tabla_versiones

C:\Users\rablo\AppData\Local\Temp\ipykernel_21232\3784026585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlms_versiones.loc[:,"version"] = "mlm"
C:\Users\rablo\AppData\Local\Temp\ipykernel_21232\3784026585.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gurobis_versiones.loc[:,"version"] = "gurobi60"
C:\Users\rablo\AppData\Local\Temp\ipykernel_21232\3784026585.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

,version,gap_best,gap_avg,avg_time,hit_sum
0,configuracion1,0.019637,0.031499,57.523142,162
1,configuracion2,0.020479,0.032576,57.520317,164
2,configuracion3,0.020095,0.031912,57.523407,158
3,configuracion4,0.019816,0.031570,57.475119,161
4,configuracion5,0.020038,0.032874,58.037029,138
5,gurobi60,0.047760,0.047760,56.064903,154
6,mlm,0.022471,0.035302,57.311257,199


In [8]:
bks["name"] = bks["name"].str.upper()
bks["fullname"] = bks["name"] + "-" + bks["Q"].astype("str") + "-" + bks["nodos"].astype("str") 
dict_bks = dict()
for i, row in bks.iterrows():
    dict_bks[row["fullname"]] = row["bks"]

In [9]:
import json
with open('bks.json', 'w') as f:
    json.dump(dict_bks, f)

In [10]:
#name = "C101-10-100"
#with open("bks.json","r") as f:
#    bks = json.loads(f.read())[name]
#    

In [11]:
bks

,name,Q,nodos,bks,fullname
0,C101,5,100,829.32,C101-5-100
1,C101,10,100,563.37,C101-10-100
2,C101,15,100,528.28,C101-15-100
3,C101,20,100,489.00,C101-20-100
4,C101,1000,100,466.52,C101-1000-100
...,...,...,...,...,...
875,RC2_2_9,15,200,1757.01,RC2_2_9-15-200
876,RC2_2_9,20,150,1335.58,RC2_2_9-20-150
877,RC2_2_9,20,200,1616.80,RC2_2_9-20-200
878,RC2_2_9,1000,150,1175.83,RC2_2_9-1000-150


In [12]:
mlms[(mlms.nodos == 100) & (mlms.Q >=20)].sort_values("gap_best", ascending = False).head(50)

,name,min,avg,t_avg,Q,nodos,tiempos,bks,best,gap_best,gap_avg,hit
2010,RC103,638.95,645.80,60.095336,20,100,60,633.32,638.95,0.008890,0.019706,0
1986,R206,633.57,635.50,60.000477,20,100,60,628.24,633.57,0.008484,0.011556,0
1842,RC103,611.40,614.29,60.063292,1000,100,60,606.71,611.40,0.007730,0.012494,0
2021,R203,623.34,627.89,60.000455,20,100,60,619.16,623.34,0.006751,0.014100,0
1996,R111,613.49,613.49,60.077301,20,100,60,609.49,613.49,0.006563,0.006563,0
1982,RC107,631.45,631.45,60.051127,20,100,60,627.94,631.45,0.005590,0.005590,0
2001,R106,656.46,658.00,60.013287,20,100,60,653.42,656.46,0.004652,0.007009,0
2017,C204,545.12,545.24,60.030675,20,100,60,543.04,545.12,0.003830,0.004051,0
2023,R104,596.23,596.23,60.045986,20,100,60,594.43,596.23,0.003028,0.003028,0
2013,R107,622.04,629.71,60.000596,20,100,60,620.36,622.04,0.002708,0.015072,0


In [13]:
versiones[(versiones.nodos == 150) & (versiones.version == "Version4")].sort_values("gap_avg", ascending = False).head(50)

NameError: name 'versiones' is not defined